# Inferentia2 (inf2.48xlarge)에서 upstage/SOLAR-10.7B-Instruct-v1.0 배치 추론 

---




# 1. 사전 필수 단계
- 아래를 클릭하셔서 사전 단계를 수행 하세요.
    - [AWS Inferentia2 설치 및 실행 가이드](Readme.md)

# 2. 배치 추론 실행

In [1]:
from vllm import LLM , SamplingParams

# 3. upstage/SOLAR-10.7B-Instruct-v1.0 모델 컴파일 후에 로딩

In [3]:
llm = LLM(
    # model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    model="upstage/SOLAR-10.7B-Instruct-v1.0",
    max_num_seqs=8,
    # The max_model_len and block_size arguments are required to be same as
    # max sequence length when targeting neuron device.
    # Currently, this is a known limitation in continuous batching support
    # in transformers-neuronx.
    # TODO(liangfu): Support paged-attention in transformers-neuronx.
    max_model_len=128,
    block_size=128,
    # The device can be automatically detected when AWS Neuron SDK is installed.
    # The device argument can be either unspecified for automated detection,
    # or explicitly assigned.
    device="neuron",
    tensor_parallel_size=2)

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

INFO 03-31 11:16:49 llm_engine.py:87] Initializing an LLM engine with config: model='upstage/SOLAR-10.7B-Instruct-v1.0', tokenizer='upstage/SOLAR-10.7B-Instruct-v1.0', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=128, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cpu, seed=0)


tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

2024-03-31 11:22:57.000433:  37636  INFO ||NEURON_CACHE||: Compile cache path: /var/tmp/neuron-compile-cache
2024-03-31 11:22:57.000481:  37636  INFO ||NEURON_CC_WRAPPER||: Call compiler with cmd: neuronx-cc compile --target=trn1 --framework=XLA /tmp/ubuntu/neuroncc_compile_workdir/0e35fee8-723a-4467-988c-68fe4f74bfa2/model.MODULE_ebbac49082b6e1cf15ac+2c2d707e.hlo_module.pb --output /tmp/ubuntu/neuroncc_compile_workdir/0e35fee8-723a-4467-988c-68fe4f74bfa2/model.MODULE_ebbac49082b6e1cf15ac+2c2d707e.neff --model-type=transformer --auto-cast=none --verbose=35
2024-03-31 11:22:57.000642:  37702  INFO ||NEURON_CACHE||: Compile cache path: /var/tmp/neuron-compile-cache
2024-03-31 11:22:57.000714:  37702  INFO ||NEURON_CC_WRAPPER||: Call compiler with cmd: neuronx-cc compile --target=trn1 --framework=XLA /tmp/ubuntu/neuroncc_compile_workdir/7e0931e0-d5c5-437a-920c-9bd9e4b4401b/model.MODULE_b3057d647b6d9d048d31+2c2d707e.hlo_module.pb --output /tmp/ubuntu/neuroncc_compile_workdir/7e0931e0-d5c5-

# 4. 모델 배치 추론

In [ ]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)


In [4]:
outputs = llm.generate(prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s]

2024-Mar-31 11:39:17.0133 36139:45147 [1] nccl_net_ofi_init:1415 CCOM WARN NET/OFI aws-ofi-nccl initialization failed
2024-Mar-31 11:39:17.0133 36139:45147 [1] init.cc:137 CCOM WARN OFI plugin initNet() failed is EFA enabled?


Processed prompts: 100%|██████████| 4/4 [00:01<00:00,  2.07it/s]

Prompt: 'Hello, my name is', Generated text: ' Sophia and I am a certified trainer and nutritionist. My passion is helping'
Prompt: 'The president of the United States is', Generated text: ' the head of state and head of government of the United States of America. The'
Prompt: 'The capital of France is', Generated text: ' Paris. It is also the most populous city in the country. Paris is'
Prompt: 'The future of AI is', Generated text: " already here, but it's not evenly distributed. While some companies have"


In [5]:
prompts = [
    "대한민국의 수도는 어디야?",
    "사과의 건강 효능에 대해서 알려줘",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)


In [6]:
outputs = llm.generate(prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 2/2 [00:00<00:00,  2.02it/s]

Prompt: '대한민국의 수도는 어디야?', Generated text: '\nWhat is the capital city of South Korea?\n\nThe capital city of'
Prompt: '사과의 건강 효능에 대해서 알려줘', Generated text: '\n\nQuestion: Please provide information about the health benefits of apples.\n'
